In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
from scipy import interpolate
from scipy.optimize import root
import obspy
import random
from cdh_mc import Model
from modal_sum import calc_nu, P_tl, P_tr, P_bl, P_br,F_tl, F_tr, F_bl, F_br

# User Defined

## Define geometry

In [6]:
params_dict = { 'min_layers':4, 'max_layers':4,  'min_rho':1, 'max_rho':3, 'min_vel':1, 'max_vel':3, \
              'min_depth':0, 'max_depth':300,} # units: N/A; g/cc; km/s; km
model = Model(params_dict)
model.layer_1.mu = 3

## Define search space

In [7]:
z = model.no_layers

nk = 1000

nyquist = 1.0*(2*np.pi) # 1 hz
nw = 64
dw = nyquist/nw

model.layer_1.upper_interface = 0 
model.layer_1.lower_interface = 4 #h1 = 4 km
model.layer_1.h = 4
model.layer_2.upper_interface = 4 
model.layer_2.lower_interface = 29 #h2 = 25 km
model.layer_2.h = 25
model.layer_3.upper_interface = 29
model.layer_3.lower_interface = 129 #h3 = 100 km
model.layer_3.h = 100

model.layer_1.velocity = 2.4*100000 # 2.4*10^5 cm/s
model.layer_2.velocity = 3.5*100000
model.layer_3.velocity = 4.5*100000
beta_max = max([model.layer_1.velocity,model.layer_2.velocity,model.layer_3.velocity])
beta_min = min([model.layer_1.velocity,model.layer_2.velocity,model.layer_3.velocity])

model.layer_1.density = 2.0 # 2 g/cm^3
model.layer_2.density = 2.67
model.layer_3.density = 3.3

model.layer_1.mu = model.layer_1.velocity**2 * model.layer_1.density
model.layer_2.mu = model.layer_2.velocity**2 * model.layer_2.density
model.layer_3.mu = model.layer_3.velocity**2 * model.layer_3.density

# Generate propogator matrix for top layer

In [9]:

B = []

for z in range(z-2):
    for w in np.arange(dw, nyquist+dw, dw):
        kmin = w/beta_max
        kmax = w/beta_min
        for k in np.arange(kmin, kmax, kmin/nk):

            current = z+1
            below = z+2
            current_layer = getattr(model, "layer_%i" % current)
            layer_below = getattr(model, "layer_%i" % below)

            F = np.array([[F_tl(calc_nu(k, w, current_layer.velocity), current_layer.h, current_layer.mu), #Fn
                           F_tr(calc_nu(k, w, current_layer.velocity), current_layer.h, current_layer.mu)], 
                          [F_bl(calc_nu(k, w, current_layer.velocity), current_layer.h, current_layer.mu), 
                           F_br(calc_nu(k, w, current_layer.velocity), current_layer.h, current_layer.mu)]]) 
    
            P = np.array([[P_tl(calc_nu(k, w, layer_below.velocity), layer_below.h), #Pn+1
                           P_tr(calc_nu(k, w, layer_below.velocity), layer_below.h, layer_below.mu)],
                          [P_bl(calc_nu(k, w, layer_below.velocity), layer_below.h, layer_below.mu), 
                           P_br(calc_nu(k, w, layer_below.velocity), layer_below.h)]]) 

            B.append(F*P)

In [34]:
b21 = [b[[1],[0]][0] for b in B]
J = np.where(np.imag(b21) == 0)[0]
a = [b21[j] for j in J]
threshold = 0.01
l = np.where(np.abs(a)< threshold)[0]
#kk = max()